In [0]:
# Imports
import numpy as np
import tensorflow as tf
import tensorflow.keras.optimizers
import tensorflow.keras.losses
import matplotlib.pyplot as plt
import seaborn as sns
import datetime
import json
import os

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [0]:
print(tf.__version__)

2.2.0


## Import and check data

In [0]:
# Mount drive to access files

from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
# Paths
train_y_path = "/content/drive/My Drive/DD2424Files/train_y.npz"
val_y_path   = "/content/drive/My Drive/DD2424Files/val_y.npz"
test_y_path  = "/content/drive/My Drive/DD2424Files/test_y.npz"

train_X_path = "/content/drive/My Drive/DD2424Files/train_X.npz"
val_X_path   = "/content/drive/My Drive/DD2424Files/val_X.npz"
test_X_path  = "/content/drive/My Drive/DD2424Files/test_X.npz"

train_X_mfcc_path = "/content/drive/My Drive/DD2424Files/train_X_mfcc.npz"
val_X_mfcc_path   = "/content/drive/My Drive/DD2424Files/val_X_mfcc.npz"
test_X_mfcc_path  = "/content/drive/My Drive/DD2424Files/test_X_mfcc.npz"

In [0]:
# Import training and validation data
train_y = np.load(train_y_path)['arr_0']
train_y = tf.one_hot(train_y, 8)

val_y = np.load(val_y_path)['arr_0']
val_y = tf.one_hot(val_y, 8)

#train_X = tf.convert_to_tensor(np.expand_dims(np.load(train_X_path)['arr_0'], -1))
#val_X = tf.convert_to_tensor(np.expand_dims(np.load(val_X_path)['arr_0'], -1))
train_X_mfcc = np.load(train_X_mfcc_path)['arr_0']
val_X_mfcc = np.load(val_X_mfcc_path)['arr_0']

In [0]:
train_X_mfcc = np.load(train_X_mfcc_path)['arr_0']
val_X_mfcc = np.load(val_X_mfcc_path)['arr_0']

In [0]:
# Check the data shape
print(train_X_mfcc.shape)
print(val_X_mfcc.shape)

(6397, 140)
(800, 140)


In [0]:
# Assert assumptions about data

MEL_BINS   = 96
TIME_STEPS = 1405

def check_data(X, y):
  assert len(y.shape) == 2
  assert len(X.shape) == 4
  assert y.shape[0] == X.shape[0]
  assert X.shape[1] == MEL_BINS
  assert X.shape[2] == TIME_STEPS
  assert X.shape[3] == 1

check_data(val_X, val_y)
check_data(train_X, train_y)

NameError: ignored

## The model

In [0]:
# All model definitions

def get_mlp_model():

  mlp_model = tf.keras.models.Sequential([
    tf.keras.layers.Flatten(),
    #tf.keras.layers.Dense(128, activation='relu'),
    #tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(8)
  ])
  return mlp_model

#-------------------------------------------------------

def cnn1_model(hyperparams):
  NUMBER_FILTERS_1 = hyperparams.get("number_filters_1")
  KERNEL_SIZE = hyperparams.get("kernel_size_1")

  model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(NUMBER_FILTERS_1, KERNEL_SIZE, padding = 'same', input_shape = (96, 1405, 1)),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(8)                             
  ])
  return model

#-------------------------------------------------------
def cnn_k1c2_model(hyperparams):
  h = hyperparams

  model = tf.keras.models.Sequential()

  # Input layer
  model.add(tf.keras.layers.InputLayer(input_shape = (96, 1405, 1)))

  # Batch normalize
  model.add(tf.keras.layers.BatchNormalization(axis = 1))

  # Convolutional blocks
  for i in range(len(h["number_filters"])):
    NUMBER_FILTERS = h["number_filters"][i]
    KERNEL_SIZE    = h["kernel_size"][i]
    POOLING_SIZE   = h["pooling_size"][i]
    DROP_OUT_SIZE  = h["dropout"]

    model.add(tf.keras.layers.Conv2D(NUMBER_FILTERS, KERNEL_SIZE, padding = 'same'))
    model.add(tf.keras.layers.BatchNormalization(axis = 3))
    model.add(tf.keras.layers.Activation("elu"))
    model.add(tf.keras.layers.MaxPool2D(POOLING_SIZE))
    model.add(tf.keras.layers.Dropout(DROP_OUT_SIZE))


  # Classifier block
  model.add(tf.keras.layers.Flatten())
  for el in h["hidden_layers"]:
    model.add(tf.keras.layers.Dense(el))
    model.add(tf.keras.layers.Dropout(h["drop_out_hidden"]))

  model.add(tf.keras.layers.Dense(8))

  return model

  #-----------------------------
def mlp_model(hyperparams):

  NO_HL = hyperparams["no_hl"]
  HL_WIDTH = hyperparams["hl_width"]
  DROPOUT = hyperparams["dropout"]

  mlp_model = tf.keras.models.Sequential()
  mlp_model.add(tf.keras.layers.Flatten())
  for i in range(NO_HL):
    mlp_model.add(tf.keras.layers.Dense(HL_WIDTH, activation='elu'))
    mlp_model.add(tf.keras.layers.Dropout(DROPOUT))
  mlp_model.add(tf.keras.layers.Dense(8, activation = 'softmax'))

  return mlp_model



In [0]:
# Utility functions for saving outputs

def plot_history(history, val_string, loss_string):
  # Plot results
  plt.figure(1)
  plt.plot(history.history['accuracy'])
  plt.plot(history.history['val_accuracy'])
  plt.xlabel('Epoch')
  plt.ylabel('Acc')
  plt.legend(['Accuracy', 'Validation accuracy'])
  plt.title('Accuracy')
  plt.ylim([0, 1])
  plt.savefig(val_string, dpi = 300)
  plt.close()

  plt.figure(2)
  plt.plot(history.history['loss'])
  plt.plot(history.history['val_loss'])
  plt.xlabel('Epoch')
  plt.ylabel('Loss')
  plt.legend(['Loss', 'Validation loss'])
  plt.title('Loss')
  plt.savefig(loss_string, dpi = 300)
  plt.close()

def filename():
	return "S" + str(datetime.datetime.now()).replace(':','.')[2:19]

In [0]:
# Run an experiment

def run_experiment(model, model_string, hyperparams):

  file_location = "/content/drive/My Drive/DD2424Files/Results/" + filename() + "/"
  os.mkdir(file_location)

  # Compile model

  optimizer = tf.keras.optimizers.Adam(learning_rate=hyperparams.get("learning_rate"), epsilon=hyperparams.get("epsilon"))
  loss = tf.keras.losses.CategoricalCrossentropy(from_logits = True)
  metrics = ['accuracy']

  model.compile(optimizer = optimizer, loss = loss, metrics = metrics)

  # Train model
  BATCH_SIZE = hyperparams.get("batch_size")
  EPOCHS = hyperparams.get("epochs")
  tf.keras.backend.clear_session()

  save_callback = tf.keras.callbacks.ModelCheckpoint(
      file_location + "models_" + model_string, monitor='val_accuracy', verbose=0, save_best_only=True,
      save_weights_only=False, mode='max', save_freq='epoch')

  #history = model.fit(train_X, train_y, batch_size=BATCH_SIZE, epochs=EPOCHS,
                          #validation_data=(val_X, val_y), verbose=1, callbacks = [save_callback])
    
  history = model.fit(train_X_mfcc, train_y, batch_size=BATCH_SIZE, epochs=EPOCHS,
                          validation_data=(val_X_mfcc, val_y), verbose=1, callbacks = [save_callback])

  # Save stuff
  plot_history(history, file_location + "acc_" + model_string + ".png", file_location + "loss_" + model_string + ".png")
  json.dump(history.history, open(file_location + "history_" + model_string, 'w'))
  json.dump(hyperparams, open(file_location + "hyperparams_" + model_string, 'w'))

  return history




In [0]:
# Experiment 1

hyperparams = {"number_filters_1" : 15,
               "kernel_size_1" : (1, 4),
               "epsilon": 0.01, 
               "learning_rate": 0.01, 
               "batch_size": 32, 
               "epochs": 20}

model_string = "cnn15-fc8"
model = cnn1_model(hyperparams)

history = run_experiment(model, model_string, hyperparams)


## Experiment 2

In [0]:
# Experiment 2

hyperparams = {"number_filters" : [15, 15, 30, 30],
               "kernel_size" : [(96, 4), (1,4), (1,4), (1,4)],
               "pooling_size" : [(96,4), (1,5), (1,8), (1,8)],
               "hidden_layers": [30,30],
               "epsilon": 0.01, 
               "learning_rate": 0.01, 
               "batch_size": 32, 
               "epochs": 100,
               "dropout" : 0,
               "drop_out_hidden" : 0
               }

model_string = "cnn_k2c1"
model = cnn_k1c2_model(hyperparams)

history = run_experiment(model, model_string, hyperparams)
model.summary()

Epoch 1/100
200/200 [==============================] - ETA: 0s - loss: 1.8332 - accuracy: 0.3186WARNING:tensorflow:From /usr/local/lib/python3.6/dist-packages/tensorflow/python/ops/resource_variable_ops.py:1817: calling BaseResourceVariable.__init__ (from tensorflow.python.ops.resource_variable_ops) with constraint is deprecated and will be removed in a future version.
Instructions for updating:
If using Keras pass *_constraint arguments to layers.
INFO:tensorflow:Assets written to: /content/drive/My Drive/DD2424Files/Results/S20-05-12 13.13.18/models_cnn_k2c1/assets
200/200 [==============================] - 79s 394ms/step - loss: 1.8332 - accuracy: 0.3186 - val_loss: 2.2714 - val_accuracy: 0.2225
Epoch 2/100
200/200 [==============================] - 77s 385ms/step - loss: 1.6329 - accuracy: 0.4041 - val_loss: 2.1222 - val_accuracy: 0.2550
Epoch 3/100
200/200 [==============================] - 77s 385ms/step - loss: 1.5572 - accuracy: 0.4472 - val_loss: 2.1937 - val_accuracy: 0.3125


## Experiment 3

In [0]:
  # Experiment 
  
  hyperparams = {
      "hl_width" : 128,
      "no_hl" : 10,
      "dropout" : 0,
      "epsilon": 0.01,
      "learning_rate": 0.01,
      "batch_size": 32,
      "epochs": 150
  }

  model_string = "mlp_10HL128"
  model = mlp_model(hyperparams)

  history = run_experiment(model, model_string, hyperparams)
  model.summary()

Epoch 1/150
200/200 [==============================] - 4s 19ms/step - loss: 2.1484 - accuracy: 0.1251 - val_loss: 2.1490 - val_accuracy: 0.1250
Epoch 2/150
200/200 [==============================] - 1s 6ms/step - loss: 2.1491 - accuracy: 0.1249 - val_loss: 2.1490 - val_accuracy: 0.1250
Epoch 3/150
200/200 [==============================] - 1s 6ms/step - loss: 2.1491 - accuracy: 0.1249 - val_loss: 2.1490 - val_accuracy: 0.1250
Epoch 4/150
200/200 [==============================] - 1s 6ms/step - loss: 2.1491 - accuracy: 0.1249 - val_loss: 2.1490 - val_accuracy: 0.1250
Epoch 5/150
200/200 [==============================] - 1s 6ms/step - loss: 2.1491 - accuracy: 0.1249 - val_loss: 2.1490 - val_accuracy: 0.1250
Epoch 6/150
200/200 [==============================] - 1s 6ms/step - loss: 2.1491 - accuracy: 0.1249 - val_loss: 2.1490 - val_accuracy: 0.1250
Epoch 7/150
200/200 [==============================] - 1s 6ms/step - loss: 2.1491 - accuracy: 0.1249 - val_loss: 2.1490 - val_accuracy: 0.125

## Try on testdata

In [0]:
# Use test data on some models
#test_X = tf.convert_to_tensor(np.expand_dims(np.load(test_X_path)['arr_0'], -1))
test_X_mfcc = np.load(test_X_mfcc_path)['arr_0']
test_y = np.load(test_y_path)['arr_0']
test_y = tf.one_hot(test_y, 8)

In [0]:
# import model and evaluate on test data

#date_and_time = "20-05-12 00.28.22"
date_and_time = "S20-05-12 17.38.19"
custom_name = "mlp_1HL128"

model_location = "/content/drive/My Drive/DD2424Files/Results/" + date_and_time + "/models_" + custom_name

my_model = tf.keras.models.load_model(model_location)


my_model.evaluate(train_X_mfcc, train_y)
my_model.evaluate(val_X_mfcc, val_y)
my_model.evaluate(test_X_mfcc, test_y)

25/25 [==============================] - 0s 2ms/step - loss: 1.9657 - accuracy: 0.3487


[1.965691328048706, 0.3487499952316284]